# Volume 2, Chapter 15: Building AI Agents - Autonomous Network Operations

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eduardd76/AI_for_networking_and_security_engineers/blob/master/Volume-2-Practical-Applications/Colab-Notebooks/Vol2_Ch15_Building_AI_Agents.ipynb)

## What You'll Learn
- Agent fundamentals and the observe-think-act-verify loop
- Building simple and looping agents
- Creating tools for agent interactions
- Network diagnostic agent implementation
- Safe operations with approvals and rollbacks
- Production deployment patterns


## Setup & Dependencies

In [ ]:
!pip install -q anthropic

import os
import json
from datetime import datetime
from typing import Dict, List, Callable
from getpass import getpass
from dataclasses import dataclass

if 'ANTHROPIC_API_KEY' not in os.environ:
    api_key = getpass('Enter your Anthropic API key: ')
    os.environ['ANTHROPIC_API_KEY'] = api_key

print('✅ Dependencies installed and API key configured!')

## The Problem: Manual vs Autonomous Operations

### Manual Process: 35 minutes
- Device goes down
- User notices / receives alert
- SSH and check status (5 min)
- Check interfaces and logs (10 min)
- Search docs for fix (10 min)
- Apply fix (5 min)
- Verify (5 min)

### Autonomous Agent: 1 minute
- Agent detects device down immediately
- Agent diagnoses issue (2 sec)
- Agent searches docs (1 sec)
- Agent proposes fix to team
- Human approves (1 min)
- Agent applies fix (1 sec)
- Agent verifies (1 sec)

**35x faster. 99% less human work.**

## The Agent Loop

```
1. OBSERVE → Get current state
2. THINK   → Reason about situation (Claude)
3. ACT     → Execute action (tools)
4. VERIFY  → Check results
Loop
```

## Part 1: Tool Framework

In [ ]:
from dataclasses import dataclass

@dataclass
class Tool:
    name: str
    description: str
    execute_func: Callable

class ToolKit:
    def __init__(self):
        self.tools = {}
    
    def register(self, tool: Tool):
        self.tools[tool.name] = tool
        print(f'✓ Registered: {tool.name}')
    
    def list_tools(self) -> str:
        return '\n'.join([f'- {name}: {tool.description}' for name, tool in self.tools.items()])
    
    def execute(self, tool_name: str, **kwargs) -> str:
        if tool_name not in self.tools:
            return f'ERROR: Unknown tool {tool_name}'
        try:
            return str(self.tools[tool_name].execute_func(**kwargs))
        except Exception as e:
            return f'ERROR: {str(e)}'

print('✅ Tool framework ready')

## Create Sample Tools

In [ ]:
DEVICE_DATABASE = {
    'router-core-01': {'status': 'up', 'bgp_enabled': True, 'ospf_enabled': True, 'ospf_neighbors': 3},
    'router-core-02': {'status': 'down', 'bgp_enabled': True, 'ospf_enabled': False, 'ospf_neighbors': 0}
}

def check_device_status(device: str) -> str:
    if device not in DEVICE_DATABASE:
        return f'Device {device} not found'
    return f'{device} is {DEVICE_DATABASE[device]["status"].upper()}'

def check_bgp_status(device: str) -> str:
    if device not in DEVICE_DATABASE:
        return f'Device {device} not found'
    db = DEVICE_DATABASE[device]
    return 'BGP is running' if db['bgp_enabled'] else 'BGP is NOT running'

def check_ospf_status(device: str) -> str:
    if device not in DEVICE_DATABASE:
        return f'Device {device} not found'
    db = DEVICE_DATABASE[device]
    if db['ospf_enabled']:
        return f'OSPF running with {db["ospf_neighbors"]} neighbors'
    return 'OSPF is NOT running - THIS IS THE PROBLEM'

def search_docs(query: str) -> str:
    if 'ospf' in query.lower():
        return 'OSPF must be enabled with router ospf 1 and network statements'
    return 'Documentation found'

toolkit = ToolKit()
toolkit.register(Tool('check_device_status', 'Check if device is up/down', check_device_status))
toolkit.register(Tool('check_bgp_status', 'Check BGP status', check_bgp_status))
toolkit.register(Tool('check_ospf_status', 'Check OSPF status', check_ospf_status))
toolkit.register(Tool('search_docs', 'Search documentation', search_docs))

## Part 2: Building Agents

In [ ]:
from anthropic import Anthropic

class SimpleNetworkAgent:
    def __init__(self, toolkit: ToolKit, api_key: str):
        self.client = Anthropic(api_key=api_key)
        self.toolkit = toolkit
    
    def help_with_task(self, task: str) -> Dict:
        print(f'\n🤖 Task: {task}')
        tools_available = self.toolkit.list_tools()
        
        prompt = f'''You are a network agent with these tools:
{tools_available}
Task: {task}
Decide which tool to use and provide the answer.'''
        
        response = self.client.messages.create(
            model='claude-sonnet-4-20250514',
            max_tokens=300,
            messages=[{'role': 'user', 'content': prompt}]
        )
        return {'answer': response.content[0].text}

class DiagnosticAgent:
    def __init__(self, toolkit: ToolKit, api_key: str, max_steps: int = 5):
        self.client = Anthropic(api_key=api_key)
        self.toolkit = toolkit
        self.max_steps = max_steps
        self.history = []
    
    def diagnose(self, problem: str) -> Dict:
        print(f'\n🔍 Diagnosing: {problem}')
        self.history = []
        
        for step in range(self.max_steps):
            action = self._decide_next_action(problem)
            
            if action.get('type') == 'DONE':
                return {'status': 'diagnosed', 'conclusion': action.get('conclusion')}
            
            tool_name = action.get('tool')
            if tool_name:
                result = self.toolkit.execute(tool_name, device=action.get('device'))
                self.history.append({'tool': tool_name, 'result': result})
                print(f'  Step {step+1}: {tool_name} → {result}')
        
        return {'status': 'max_steps'}
    
    def _decide_next_action(self, problem: str) -> Dict:
        tools_available = self.toolkit.list_tools()
        history_str = '\n'.join([f"- {h['tool']}: {h['result'][:50]}" for h in self.history])
        
        prompt = f'''Diagnose this problem:
{problem}

Tools:
{tools_available}

Progress:
{history_str if history_str else "(just started)"}

Next action - return JSON with type (TOOL or DONE), tool name (if TOOL), device name (if needed), or conclusion (if DONE).'''
        
        response = self.client.messages.create(
            model='claude-sonnet-4-20250514',
            max_tokens=200,
            messages=[{'role': 'user', 'content': prompt}]
        )
        
        try:
            return json.loads(response.content[0].text)
        except:
            return {'type': 'DONE', 'conclusion': response.content[0].text}

print('✅ Agent classes loaded')

## Test the Agents

In [ ]:
agent = SimpleNetworkAgent(toolkit, os.environ['ANTHROPIC_API_KEY'])
result1 = agent.help_with_task('Check if router-core-01 is up')
print(f'Result:\n{result1["answer"]}')

diagnostic = DiagnosticAgent(toolkit, os.environ['ANTHROPIC_API_KEY'])
result2 = diagnostic.diagnose('Router-core-02 is down. Why?')
print(f'\nDiagnosis: {result2}')

## Best Practices

### Safe Operations
- Always request approval before changes
- Implement rollback capability
- Comprehensive logging of all actions
- Limit agent scope and permissions
- Rate limiting (max changes/hour)

### Integration
- Use Chapter 13 documentation
- Leverage Chapter 14 RAG for searches
- Combine observation + reasoning + action
- Verify fixes before reporting success

## Summary

### You've Built
- Tool framework for agent capabilities
- Simple agents for single decisions
- Diagnostic agents for complex troubleshooting
- Safe operation patterns

### Impact
- Issue detection: Automatic (not manual)
- Time to diagnose: 1-2 min (vs 30-45 min)
- Time to fix: <1 min (vs 15-30 min)
- **Total MTTR: 2-5 min (vs 60-100 min)**

### You Now Have
- Ch13: Auto-generated documentation
- Ch14: Searchable documentation (RAG)
- Ch15: Autonomous troubleshooting agents

**Complete autonomous network operations platform! 🚀**